In [1]:
import random as rnd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import mxnet as mx
from mxnet import gluon, nd, autograd

In [3]:
range_start = 0
range_end = 299

In [31]:
num_inputs = 2
num_outputs = 1
num_examples = 10000
model_ctx = mx.cpu()

def real_fn(X):
    return 2 * X[:, 0] - 3.4 * X[:, 1] + 4.2
    
X = nd.random_normal(shape=(num_examples, num_inputs))
noise = 0.01 * nd.random_normal(shape=(num_examples,))
y = real_fn(X) + noise


In [32]:
batch_size = 4
train_data = gluon.data.DataLoader(gluon.data.ArrayDataset(X, y),
                                      batch_size=batch_size, shuffle=True)

In [33]:
net = gluon.nn.Dense(1)

In [34]:
net.initialize(mx.init.Normal(sigma=1.), ctx=model_ctx, force_reinit=True)


In [35]:
square_loss = gluon.loss.L2Loss()
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.0001})


In [36]:
epochs = 10
loss_sequence = []
num_batches = num_examples / batch_size

for e in range(epochs):
    cumulative_loss = 0
    # inner loop
    for i, (data, label) in enumerate(train_data):
        data = data.as_in_context(model_ctx)
        label = label.as_in_context(model_ctx)
        with autograd.record():
            output = net(data)
            loss = square_loss(output, label)
        loss.backward()
        trainer.step(batch_size)
        cumulative_loss += nd.mean(loss).asscalar()
        net.save_parameters('model.params')
    print("Epoch %s, loss: %s" % (e, cumulative_loss / num_examples))
    loss_sequence.append(cumulative_loss)

Epoch 0, loss: 3.4170194335341453
Epoch 1, loss: 2.0791151904396714
Epoch 2, loss: 1.2651153823085128
Epoch 3, loss: 0.7698360557321459
Epoch 4, loss: 0.4684779713962227
Epoch 5, loss: 0.28510312656909226
Epoch 6, loss: 0.17351538671012967
Epoch 7, loss: 0.10560934834857472
Epoch 8, loss: 0.06428302924255841
Epoch 9, loss: 0.03913161084465683


In [37]:
infer_net = gluon.nn.Dense(1)
infer_net.load_parameters('model.params')

In [38]:
X


[[-1.9173642  -1.680917  ]
 [-2.6438675   1.8643172 ]
 [ 0.4498908  -1.701221  ]
 ...
 [ 0.70534587  0.9442211 ]
 [-1.2200671  -0.69724506]
 [ 0.97108454  1.5380151 ]]
<NDArray 10000x2 @cpu(0)>

In [50]:
def moving_average(a, n=3) :
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n